In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for the plots
sns.set_palette("viridis")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams['axes.grid'] = True

In [9]:
# Load the external dataset
def load_data(filepath, name):
    df = pd.read_csv(filepath)
    print(f"{name} data loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
    return df

external = load_data(r"d:\GitHub\calcifer\data\interim\cleaned_external_data.csv", "external")

external data loaded successfully with 2029 rows and 3 columns.


In [10]:
# Provide a structured overview to detect potential anomalies that could impact further analysis
def dataframe_summary(df: pd.DataFrame) -> pd.DataFrame:
    summary = pd.DataFrame({
        "Data Type": df.dtypes,
        "Non-Null Count": df.notnull().sum(),
        "Missing Values": df.isnull().sum(),
        "Distinct Values": df.nunique(),
        "Zeros Count": (df == 0).sum(),
        "Negative Values": (df < 0).sum(),
        "Infinite Values": np.isinf(df).sum()
    })

    print(summary)

In [11]:
# Get the descriptive statistics
external.describe()

,Tg,Ta,Tc
count,2029.000000,2029.000000,2029.000000
mean,34.368379,24.107541,37.029428
std,0.738554,1.324160,0.510204
min,30.225000,20.200000,35.540000
25%,33.955000,23.400000,36.790000
50%,34.387500,24.000000,36.940000
75%,34.787500,24.700000,37.140000
max,37.693333,29.100000,40.340000


In [12]:
# Get the detailed summary
dataframe_summary(external)

   Data Type  Non-Null Count  Missing Values  Distinct Values  Zeros Count  \
Tg   float64            2029               0             1215            0   
Ta   float64            2029               0               78            0   
Tc   float64            2029               0               70            0   

    Negative Values  Infinite Values  
Tg                0                0  
Ta                0                0  
Tc                0                0  
